# Submission - Inquisitive_Turtles

In [2]:
import kagglehub
path = kagglehub.dataset_download("schran/insurance-premium-prediction")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/insurance-premium-prediction


In [94]:
import pandas as pd
df = pd.read_csv('/kaggle/input/insurance-premium-prediction/Insurance Premium Prediction Dataset.csv')
print(df.columns,df.dtypes)

Index(['Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Premium Amount',
       'Policy Start Date', 'Customer Feedback', 'Smoking Status',
       'Exercise Frequency', 'Property Type'],
      dtype='object') Age                     float64
Gender                   object
Annual Income           float64
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score            float64
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age               int64
Credit Score            float64
Insurance Duration        int64
Premium Amount          float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency

In [95]:
df.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,274175.000000,264905.000000,250974.000000,268263.000000,197572.000000,278860.000000,250974.000000,278860.000000,277019.000000
mean,41.020771,42089.085329,1.998048,28.584290,0.998117,9.520283,574.362049,5.007764,966.118667
std,13.549683,35444.517255,1.412312,15.966208,1.000795,5.767915,158.792037,2.581349,909.404567
min,18.000000,0.000000,0.000000,0.035436,0.000000,0.000000,300.000000,1.000000,0.000000
25%,29.000000,13588.000000,1.000000,16.149890,0.000000,5.000000,437.000000,3.000000,286.000000
50%,41.000000,32191.000000,2.000000,26.451244,1.000000,10.000000,575.000000,5.000000,688.000000
75%,53.000000,62164.000000,3.000000,38.966369,2.000000,15.000000,712.000000,7.000000,1367.000000
max,64.000000,149997.000000,4.000000,93.876090,9.000000,19.000000,849.000000,9.000000,4999.000000


In [96]:
missing_values = df.isnull().sum()
cols_with_missing = missing_values[missing_values > 0]
print(cols_with_missing)

Age                      4685
Annual Income           13955
Marital Status           5019
Number of Dependents    27886
Occupation              81288
Health Score            10597
Previous Claims         81288
Credit Score            27886
Premium Amount           1841
Customer Feedback       18349
dtype: int64


Getting rid of null values in target column

In [97]:
df = df.dropna(subset = 'Premium Amount')

In [98]:
missing_values = df.isnull().sum()
cols_with_missing = missing_values[missing_values > 0]
print(cols_with_missing)

Age                      4658
Annual Income           13851
Marital Status           4991
Number of Dependents    27689
Occupation              80748
Health Score            10525
Previous Claims         80749
Credit Score            27700
Customer Feedback       18258
dtype: int64


In [99]:
df.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,272361.000000,263168.000000,249330.000000,266494.000000,196270.000000,277019.000000,249319.000000,277019.000000,277019.000000
mean,41.020139,42095.013159,1.998267,28.583739,0.998105,9.519318,574.389365,5.007898,966.118667
std,13.548304,35443.338341,1.412300,15.964010,1.000770,5.767438,158.794989,2.581473,909.404567
min,18.000000,0.000000,0.000000,0.035436,0.000000,0.000000,300.000000,1.000000,0.000000
25%,29.000000,13591.000000,1.000000,16.149422,0.000000,5.000000,437.000000,3.000000,286.000000
50%,41.000000,32207.000000,2.000000,26.448428,1.000000,10.000000,575.000000,5.000000,688.000000
75%,53.000000,62176.500000,3.000000,38.964028,2.000000,15.000000,712.000000,7.000000,1367.000000
max,64.000000,149997.000000,4.000000,93.876090,9.000000,19.000000,849.000000,9.000000,4999.000000


In [100]:
Q1 = df['Previous Claims'].quantile(0.25)
Q3 = df['Previous Claims'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_cleaned = df[(df['Previous Claims'] >= lower_bound) & (df['Previous Claims'] <= upper_bound)]
df_outliers = df[(df['Previous Claims'] < lower_bound) | (df['Previous Claims'] > upper_bound)]

print('number of outliers: ',df_outliers['Previous Claims'].count())
print("Original DataFrame with outliers:",df['Previous Claims'].count())
print("DataFrame after removing outliers:",df_cleaned['Previous Claims'].count())

number of outliers:  117
Original DataFrame with outliers: 196270
DataFrame after removing outliers: 196153


In [101]:
df_cleaned.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,192857.000000,186380.000000,176596.000000,188706.000000,196153.000000,196153.000000,176456.000000,196153.000000,196153.000000
mean,41.002276,42048.496631,1.999219,28.569734,0.994973,9.507670,574.588645,5.011608,965.925660
std,13.540229,35408.827384,1.412334,15.967410,0.992727,5.767753,158.823202,2.581916,908.763664
min,18.000000,0.000000,0.000000,0.035436,0.000000,0.000000,300.000000,1.000000,0.000000
25%,29.000000,13616.750000,1.000000,16.133740,0.000000,4.000000,437.000000,3.000000,287.000000
50%,41.000000,32167.000000,2.000000,26.424062,1.000000,10.000000,575.000000,5.000000,688.000000
75%,53.000000,62015.750000,3.000000,38.909943,2.000000,15.000000,712.000000,7.000000,1365.000000
max,64.000000,149997.000000,4.000000,93.876090,5.000000,19.000000,849.000000,9.000000,4999.000000


In [102]:
mean_imputation_cols = ['Age','Number of Dependents','Credit Score','Credit Score']

In [103]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

df_cleaned[mean_imputation_cols]= imputer.fit_transform(df_cleaned[mean_imputation_cols])
print(df_cleaned)



         Age  Gender  Annual Income Marital Status  Number of Dependents  \
2       32.0  Female        30154.0       Divorced                   3.0   
3       60.0  Female        48371.0       Divorced                   0.0   
5       38.0    Male        31769.0        Married                   1.0   
6       56.0  Female        26209.0         Single                   1.0   
7       36.0    Male         8762.0         Single                   3.0   
...      ...     ...            ...            ...                   ...   
278851  25.0  Female         4630.0       Divorced                   4.0   
278852  47.0  Female        14001.0         Single                   0.0   
278853  28.0  Female        74931.0        Married                   0.0   
278854  63.0    Male        34311.0       Divorced                   0.0   
278855  53.0    Male        22594.0       Divorced                   4.0   

       Education Level     Occupation  Health Score  Location    Policy Type  \
2      

<ipython-input-103-a269f45f3bbf>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[mean_imputation_cols]= imputer.fit_transform(df_cleaned[mean_imputation_cols])


In [104]:
df_cleaned.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,196153.000000,186380.000000,196153.000000,188706.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000
mean,41.002276,42048.496631,1.999219,28.569734,0.994973,9.507670,574.588645,5.011608,965.925660
std,13.425987,35408.827384,1.340079,15.967410,0.992727,5.767753,150.638007,2.581916,908.763664
min,18.000000,0.000000,0.000000,0.035436,0.000000,0.000000,300.000000,1.000000,0.000000
25%,30.000000,13616.750000,1.000000,16.133740,0.000000,4.000000,452.000000,3.000000,287.000000
50%,41.000000,32167.000000,2.000000,26.424062,1.000000,10.000000,574.588645,5.000000,688.000000
75%,53.000000,62015.750000,3.000000,38.909943,2.000000,15.000000,697.000000,7.000000,1365.000000
max,64.000000,149997.000000,4.000000,93.876090,5.000000,19.000000,849.000000,9.000000,4999.000000


In [105]:
missing_values = df_cleaned.isnull().sum()
check_missing = missing_values[missing_values > 0]
print(check_missing)

Annual Income         9773
Marital Status        3490
Occupation           57166
Health Score          7447
Customer Feedback    12959
dtype: int64


In [106]:
mode_imputation_cols=['Annual Income','Health Score']

In [107]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')

df_cleaned[mode_imputation_cols]= imputer.fit_transform(df_cleaned[mode_imputation_cols])
print(df_cleaned)


         Age  Gender  Annual Income Marital Status  Number of Dependents  \
2       32.0  Female        30154.0       Divorced                   3.0   
3       60.0  Female        48371.0       Divorced                   0.0   
5       38.0    Male        31769.0        Married                   1.0   
6       56.0  Female        26209.0         Single                   1.0   
7       36.0    Male         8762.0         Single                   3.0   
...      ...     ...            ...            ...                   ...   
278851  25.0  Female         4630.0       Divorced                   4.0   
278852  47.0  Female        14001.0         Single                   0.0   
278853  28.0  Female        74931.0        Married                   0.0   
278854  63.0    Male        34311.0       Divorced                   0.0   
278855  53.0    Male        22594.0       Divorced                   4.0   

       Education Level     Occupation  Health Score  Location    Policy Type  \
2      

<ipython-input-107-3e9759c0bb16>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[mode_imputation_cols]= imputer.fit_transform(df_cleaned[mode_imputation_cols])


In [108]:
missing_values = df_cleaned.isnull().sum()
check_missing = missing_values[missing_values > 0]
print(check_missing)

Marital Status        3490
Occupation           57166
Customer Feedback    12959
dtype: int64


In [109]:
mode_imputation_cols=['Marital Status','Occupation']

In [110]:
df_cleaned.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000
mean,41.002276,40069.837102,1.999219,27.486421,0.994973,9.507670,574.588645,5.011608,965.925660
std,13.425987,35580.637143,1.340079,16.583628,0.992727,5.767753,150.638007,2.581916,908.763664
min,18.000000,0.000000,0.000000,0.035436,0.000000,0.000000,300.000000,1.000000,0.000000
25%,30.000000,11221.000000,1.000000,14.888263,0.000000,4.000000,452.000000,3.000000,287.000000
50%,41.000000,29925.000000,2.000000,25.592391,1.000000,10.000000,574.588645,5.000000,688.000000
75%,53.000000,59886.000000,3.000000,38.313994,2.000000,15.000000,697.000000,7.000000,1365.000000
max,64.000000,149997.000000,4.000000,93.876090,5.000000,19.000000,849.000000,9.000000,4999.000000


In [111]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')

df_cleaned[mode_imputation_cols]= imputer.fit_transform(df_cleaned[mode_imputation_cols])
print(df_cleaned)


         Age  Gender  Annual Income Marital Status  Number of Dependents  \
2       32.0  Female        30154.0       Divorced                   3.0   
3       60.0  Female        48371.0       Divorced                   0.0   
5       38.0    Male        31769.0        Married                   1.0   
6       56.0  Female        26209.0         Single                   1.0   
7       36.0    Male         8762.0         Single                   3.0   
...      ...     ...            ...            ...                   ...   
278851  25.0  Female         4630.0       Divorced                   4.0   
278852  47.0  Female        14001.0         Single                   0.0   
278853  28.0  Female        74931.0        Married                   0.0   
278854  63.0    Male        34311.0       Divorced                   0.0   
278855  53.0    Male        22594.0       Divorced                   4.0   

       Education Level     Occupation  Health Score  Location    Policy Type  \
2      

<ipython-input-111-3e9759c0bb16>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[mode_imputation_cols]= imputer.fit_transform(df_cleaned[mode_imputation_cols])


In [112]:
missing_values = df_cleaned.isnull().sum()
check_missing = missing_values[missing_values > 0]
print(check_missing)

Customer Feedback    12959
dtype: int64


In [113]:
df_cleaned['Customer Feedback']= df_cleaned['Customer Feedback'].fillna('')

<ipython-input-113-1308beb94693>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Customer Feedback']= df_cleaned['Customer Feedback'].fillna('')


In [114]:
missing_values = df_cleaned.isnull().sum()
check_missing = missing_values[missing_values > 0]
print(check_missing)

Series([], dtype: int64)


In [115]:
df_cleaned.dtypes

Age                     float64
Gender                   object
Annual Income           float64
Marital Status           object
Number of Dependents    float64
Education Level          object
Occupation               object
Health Score            float64
Location                 object
Policy Type              object
Previous Claims         float64
Vehicle Age               int64
Credit Score            float64
Insurance Duration        int64
Premium Amount          float64
Policy Start Date        object
Customer Feedback        object
Smoking Status           object
Exercise Frequency       object
Property Type            object
dtype: object

In [116]:
df_cleaned.shape

(196153, 20)

In [117]:
df_cleaned.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000
mean,41.002276,40069.837102,1.999219,27.486421,0.994973,9.507670,574.588645,5.011608,965.925660
std,13.425987,35580.637143,1.340079,16.583628,0.992727,5.767753,150.638007,2.581916,908.763664
min,18.000000,0.000000,0.000000,0.035436,0.000000,0.000000,300.000000,1.000000,0.000000
25%,30.000000,11221.000000,1.000000,14.888263,0.000000,4.000000,452.000000,3.000000,287.000000
50%,41.000000,29925.000000,2.000000,25.592391,1.000000,10.000000,574.588645,5.000000,688.000000
75%,53.000000,59886.000000,3.000000,38.313994,2.000000,15.000000,697.000000,7.000000,1365.000000
max,64.000000,149997.000000,4.000000,93.876090,5.000000,19.000000,849.000000,9.000000,4999.000000


# One Hot encoding

In [118]:
from sklearn.preprocessing import OneHotEncoder
encoding_cols=['Customer Feedback','Occupation','Education Level','Marital Status',
               'Property Type','Exercise Frequency','Smoking Status','Policy Type','Gender','Location']


encoder = OneHotEncoder(sparse=False)

#Fit and transform the encoding columns
encoded_array = encoder.fit_transform(df_cleaned[encoding_cols])

#Convert the encoded array to a DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(encoding_cols))

# Drop original columns and concatenate encoded columns
df_cleaned = pd.concat([df_cleaned.drop(columns=encoding_cols), encoded_df], axis=1)

# Display the result
print(df_cleaned)

df_cleaned.columns



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


         Age  Annual Income  Number of Dependents  Health Score  \
2       32.0        30154.0                   3.0     14.714909   
3       60.0        48371.0                   0.0     25.346926   
5       38.0        31769.0                   1.0     14.736611   
6       56.0        26209.0                   1.0     33.238300   
7       36.0         8762.0                   3.0     13.887880   
...      ...            ...                   ...           ...   
196137   NaN            NaN                   NaN           NaN   
196138   NaN            NaN                   NaN           NaN   
196139   NaN            NaN                   NaN           NaN   
196142   NaN            NaN                   NaN           NaN   
196146   NaN            NaN                   NaN           NaN   

        Previous Claims  Vehicle Age  Credit Score  Insurance Duration  \
2                   2.0         16.0         652.0                 8.0   
3                   1.0         11.0         33

Index(['Age', 'Annual Income', 'Number of Dependents', 'Health Score',
       'Previous Claims', 'Vehicle Age', 'Credit Score', 'Insurance Duration',
       'Premium Amount', 'Policy Start Date', 'Customer Feedback_',
       'Customer Feedback_Average', 'Customer Feedback_Good',
       'Customer Feedback_Poor', 'Occupation_Employed',
       'Occupation_Self-Employed', 'Occupation_Unemployed',
       'Education Level_Bachelor's', 'Education Level_High School',
       'Education Level_Master's', 'Education Level_PhD',
       'Marital Status_Divorced', 'Marital Status_Married',
       'Marital Status_Single', 'Property Type_Apartment',
       'Property Type_Condo', 'Property Type_House',
       'Exercise Frequency_Daily', 'Exercise Frequency_Monthly',
       'Exercise Frequency_Rarely', 'Exercise Frequency_Weekly',
       'Smoking Status_No', 'Smoking Status_Yes', 'Policy Type_Basic',
       'Policy Type_Comprehensive', 'Policy Type_Premium', 'Gender_Female',
       'Gender_Male', 'Locati

In [119]:
df_cleaned.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount,Customer Feedback_,...,Smoking Status_No,Smoking Status_Yes,Policy Type_Basic,Policy Type_Comprehensive,Policy Type_Premium,Gender_Female,Gender_Male,Location_Rural,Location_Suburban,Location_Urban
count,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,...,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000,196153.000000
mean,41.002276,40069.837102,1.999219,27.486421,0.994973,9.507670,574.588645,5.011608,965.925660,0.066066,...,0.498738,0.501262,0.333755,0.331578,0.334667,0.498993,0.501007,0.331552,0.335488,0.332959
std,13.425987,35580.637143,1.340079,16.583628,0.992727,5.767753,150.638007,2.581916,908.763664,0.248398,...,0.500000,0.500000,0.471555,0.470781,0.471875,0.500000,0.500000,0.470772,0.472162,0.471273
min,18.000000,0.000000,0.000000,0.035436,0.000000,0.000000,300.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,11221.000000,1.000000,14.888263,0.000000,4.000000,452.000000,3.000000,287.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,41.000000,29925.000000,2.000000,25.592391,1.000000,10.000000,574.588645,5.000000,688.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,53.000000,59886.000000,3.000000,38.313994,2.000000,15.000000,697.000000,7.000000,1365.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,64.000000,149997.000000,4.000000,93.876090,5.000000,19.000000,849.000000,9.000000,4999.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [120]:
df_cleaned.head(5)

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount,Policy Start Date,...,Smoking Status_No,Smoking Status_Yes,Policy Type_Basic,Policy Type_Comprehensive,Policy Type_Premium,Gender_Female,Gender_Male,Location_Rural,Location_Suburban,Location_Urban
2,32.0,30154.0,3.0,14.714909,2.0,16.0,652.0,8.0,849.0,2023-11-26 15:21:39.078837,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,60.0,48371.0,0.0,25.346926,1.0,11.0,330.0,7.0,927.0,2023-02-27 15:21:39.078837,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
5,38.0,31769.0,1.0,14.736611,1.0,9.0,521.0,7.0,4016.0,2020-09-16 15:21:39.078837,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
6,56.0,26209.0,1.0,33.238300,0.0,10.0,424.0,5.0,828.0,2021-03-29 15:21:39.078837,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
7,36.0,8762.0,3.0,13.887880,1.0,15.0,641.0,9.0,1678.0,2022-12-21 15:21:39.078837,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [121]:
df_cleaned.dtypes

Age                            float64
Annual Income                  float64
Number of Dependents           float64
Health Score                   float64
Previous Claims                float64
Vehicle Age                    float64
Credit Score                   float64
Insurance Duration             float64
Premium Amount                 float64
Policy Start Date               object
Customer Feedback_             float64
Customer Feedback_Average      float64
Customer Feedback_Good         float64
Customer Feedback_Poor         float64
Occupation_Employed            float64
Occupation_Self-Employed       float64
Occupation_Unemployed          float64
Education Level_Bachelor's     float64
Education Level_High School    float64
Education Level_Master's       float64
Education Level_PhD            float64
Marital Status_Divorced        float64
Marital Status_Married         float64
Marital Status_Single          float64
Property Type_Apartment        float64
Property Type_Condo      

In [122]:
df_cleaned = df_cleaned.dropna()

In [123]:
df_cleaned.describe()

,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount,Customer Feedback_,...,Smoking Status_No,Smoking Status_Yes,Policy Type_Basic,Policy Type_Comprehensive,Policy Type_Premium,Gender_Female,Gender_Male,Location_Rural,Location_Suburban,Location_Urban
count,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,...,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000,137937.000000
mean,40.988243,40108.399197,1.999313,27.475101,0.996781,9.500192,574.596997,5.009635,965.759484,0.065929,...,0.499953,0.500047,0.333747,0.332268,0.333986,0.499402,0.500598,0.331905,0.335559,0.332536
std,13.421230,35594.487154,1.340000,16.578781,0.993894,5.768777,150.311954,2.581906,908.351252,0.248158,...,0.500002,0.500002,0.471552,0.471028,0.471636,0.500001,0.500001,0.470899,0.472187,0.471124
min,18.000000,0.000000,0.000000,0.035436,0.000000,0.000000,300.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,11233.000000,1.000000,14.858647,0.000000,4.000000,453.000000,3.000000,287.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,41.000000,30004.000000,2.000000,25.576818,1.000000,10.000000,574.588645,5.000000,689.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,53.000000,59897.000000,3.000000,38.324267,2.000000,15.000000,697.000000,7.000000,1364.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,64.000000,149997.000000,4.000000,93.876090,5.000000,19.000000,849.000000,9.000000,4999.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [35]:
missing_values = df_cleaned.isnull().sum()
check_missing = missing_values[missing_values > 0]
print(check_missing)

Series([], dtype: int64)


In [36]:
df_cleaned.dtypes

Age                            float64
Annual Income                  float64
Number of Dependents           float64
Health Score                   float64
Previous Claims                float64
Vehicle Age                    float64
Credit Score                   float64
Insurance Duration             float64
Premium Amount                 float64
Policy Start Date               object
Customer Feedback_             float64
Customer Feedback_Average      float64
Customer Feedback_Good         float64
Customer Feedback_Poor         float64
Occupation_Employed            float64
Occupation_Self-Employed       float64
Occupation_Unemployed          float64
Education Level_Bachelor's     float64
Education Level_High School    float64
Education Level_Master's       float64
Education Level_PhD            float64
Marital Status_Divorced        float64
Marital Status_Married         float64
Marital Status_Single          float64
Property Type_Apartment        float64
Property Type_Condo      

In [37]:
df_cleaned['Number of Dependents'].unique()

array([3.        , 0.        , 1.        , 4.        , 2.        ,
       1.99921856])

In [44]:
df_cleaned['Number of Dependents'] = df_cleaned['Number of Dependents'].astype('int32')

In [61]:
df_cleaned['Policy Start Date'] = df_cleaned['Policy Start Date'].astype('datetime64[ns]')

In [62]:
df_cleaned.dtypes

Age                                   float64
Annual Income                         float64
Number of Dependents                    int32
Health Score                          float64
Previous Claims                       float64
Vehicle Age                           float64
Credit Score                          float64
Insurance Duration                    float64
Premium Amount                        float64
Policy Start Date              datetime64[ns]
Customer Feedback_                    float64
Customer Feedback_Average             float64
Customer Feedback_Good                float64
Customer Feedback_Poor                float64
Occupation_Employed                   float64
Occupation_Self-Employed              float64
Occupation_Unemployed                 float64
Education Level_Bachelor's            float64
Education Level_High School           float64
Education Level_Master's              float64
Education Level_PhD                   float64
Marital Status_Divorced           

In [76]:
result = df_cleaned[np.isclose(df_cleaned['Customer Feedback_'],1.)]
print(result['Customer Feedback_'].count())

9094


In [77]:
df_cleaned.count()

Age                            137937
Annual Income                  137937
Number of Dependents           137937
Health Score                   137937
Previous Claims                137937
Vehicle Age                    137937
Credit Score                   137937
Insurance Duration             137937
Premium Amount                 137937
Policy Start Date              137937
Customer Feedback_             137937
Customer Feedback_Average      137937
Customer Feedback_Good         137937
Customer Feedback_Poor         137937
Occupation_Employed            137937
Occupation_Self-Employed       137937
Occupation_Unemployed          137937
Education Level_Bachelor's     137937
Education Level_High School    137937
Education Level_Master's       137937
Education Level_PhD            137937
Marital Status_Divorced        137937
Marital Status_Married         137937
Marital Status_Single          137937
Property Type_Apartment        137937
Property Type_Condo            137937
Property Typ